# Các thư viện cần thiết:

In [36]:
import re
import math
import datetime
import pandas as pd
from fuzzywuzzy import fuzz, process
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains

# Khởi tạo Selenium:

In [37]:
# Đường dẫn đến ChromeDriver
driver_path = r'.\\chromedriver-win64\\chromedriver.exe'

# Đường dẫn đến tệp thực thi của Google Chrome
chrome_path = r'C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe'

# Cấu hình tùy chọn Chrome
options = webdriver.ChromeOptions()
options.binary_location = chrome_path
# options.add_argument('headless')  # Chạy chế độ headless (không hiển thị giao diện)

# Tạo đối tượng Service để chỉ định đường dẫn ChromeDriver
service = Service(driver_path)

# Khởi tạo ChromeDriver với Service và các tùy chọn
driver = webdriver.Chrome(service=service, options=options)

# Tạo dataframe để lưu các việc làm thu thập được

In [38]:
# Khởi tạo DataFrame trống với các cột
columns = ['Trang thu thập', 'Tên công ty', 'Tên công việc', 'Vị trí ứng tuyển', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm',
           'Địa điểm', 'Ngày đăng tuyển', 'Lương tối thiểu', 'Lương tối đa', 'Lương TB']
df = pd.DataFrame(columns=columns)

# Thiết lập kiểu dữ liệu phù hợp
df['Ngày đăng tuyển'] = pd.to_datetime(df['Ngày đăng tuyển'], errors='coerce')
df['Lương tối thiểu'] = pd.to_numeric(df['Lương tối thiểu'], errors='coerce')
df['Lương tối đa'] = pd.to_numeric(df['Lương tối đa'], errors='coerce')
df['Lương TB'] = pd.to_numeric(df['Lương TB'], errors='coerce')

# Lọc các thông tin thu thập

## Tên công việc

In [39]:
# Từ điển từ khóa ánh xạ tên công việc
job_keywords = {
    'Lập trình viên': ['lập trình', 'developer', 'programmer', 'coder', 'software developer', 'engineer', 'dev', 
                       'backend developer', 'frontend developer', 'fullstack developer', 'software coder', 'software engineer', 
                       'python developer', 'java developer', 'c++ developer', 'mobile developer', 'golang developer', 'backend dev', 
                       'frontend dev', 'devops engineer', 'web programmer'],
    
    'Kinh doanh': ['sales', 'marketing', 'media', 'quảng cáo', 'kinh doanh', 'business', 'advertising', 'thương hiệu', 
                   'brand manager', 'digital marketing', 'sales executive', 'account manager', 'bán hàng', 'business development', 
                   'salesman', 'trưởng phòng kinh doanh', 'quản lý bán hàng', 'trade marketing', 'PR', 'quan hệ công chúng'],
    
    'Quản trị viên hệ thống': ['administrator', 'system admin', 'sysadmin', 'quản trị', 'backend', 'system manager', 'quản lí hệ thống',
                               'system engineer', 'infrastructure engineer', 'windows admin', 'linux admin', 'system analyst', 
                               'server administrator', 'cloud administrator', 'it administrator'],
    
    'Nhà phát triển web': ['web developer', 'phát triển web', 'thiết kế web', 'frontend', 'fullstack', 'web designer', 
                           'frontend engineer', 'fullstack engineer', 'wordpress developer', 'html developer', 'css developer', 
                           'javascript developer', 'web engineer', 'ui developer', 'web programmer', 'front-end', 'back-end', 
                           'react developer', 'angular developer'],
    
    'Nhà thiết kế đồ họa': ['graphic designer', 'thiết kế đồ họa', 'graphic', 'artist', 'multimedia designer', 'graphics artist', 
                            'visual designer', 'photoshop expert', 'illustrator', 'creative designer', 'branding designer', 
                            'ui artist', 'motion designer', '2d artist', '3d designer', 'digital artist', 'poster designer'],
    
    'Nhà khoa học dữ liệu': ['data scientist', 'khoa học dữ liệu', 'machine learning scientist', 'data researcher', 
                             'ml scientist', 'ai scientist', 'data science specialist', 'deep learning scientist', 
                             'data exploration', 'data analytics scientist'],
    
    'Nhà phân tích dữ liệu': ['data analyst', 'phân tích dữ liệu', 'business analyst', 'analyst', 'data analytics', 
                              'reporting analyst', 'data specialist', 'bi analyst', 'sql analyst', 'excel analyst', 
                              'business intelligence', 'data visualization', 'dashboard specialist', 'power bi developer'],
    
    'Nhà phát triển phần mềm': ['software developer', 'phát triển phần mềm', 'app developer', 'java developer', 'fullstack', 
                                'software architect', 'mobile application developer', 'desktop application developer', 
                                'application engineer', 'software creator', 'api developer', 'software solutions'],
    
    'Nhà thiết kế giao diện người dùng': ['ui designer', 'ux designer', 'giao diện', 'user interface', 'ui/ux', 'ux/ui', 
                                         'user experience', 'interaction designer', 'ui developer', 'ux researcher', 
                                         'product designer', 'ux writer', 'wireframe designer', 'prototype designer'],
    
    'Chuyên viên tư vấn CNTT': ['it consultant', 'tư vấn', 'brse', 'solution consultant', 'giải pháp', 'it advisory', 
                               'technical consultant', 'system consultant', 'it solutions expert', 'business it consultant'],
    
    'Chuyên viên hỗ trợ kỹ thuật': ['technical support', 'hỗ trợ kỹ thuật', 'it support', 'helpdesk', 'nhân viên kỹ thuật', 
                                    'camera', 'bảo hành', 'tech support', 'desktop support', 'it technician', 'support engineer', 
                                    'pc repair', 'network support', 'hardware technician'],
    
    'Chuyên viên bảo mật thông tin': ['information security', 'bảo mật', 'cybersecurity', 'an ninh', 'security engineer', 
                                     'it security', 'network security', 'security analyst', 'pentester', 'security operations', 
                                     'threat analyst', 'malware analyst'],
    
    'Kỹ sư mạng': ['network engineer', 'kỹ sư mạng', 'networking', 'network admin', 'network architect', 'wireless engineer', 
                   'network technician', 'ccna', 'network administrator', 'wan engineer', 'lan specialist', 'network infrastructure'],
    
    'Kỹ sư học máy': ['machine learning engineer', 'kỹ sư học máy', 'ml engineer', 'machine learning dev', 'ai developer', 
                      'ml architect', 'ai specialist', 'machine learning scientist', 'deep learning engineer', 'model developer'],
    
    'Kỹ sư đám mây': ['cloud engineer', 'kỹ sư đám mây', 'cloud architect', 'aws engineer', 'azure engineer', 'gcp engineer', 
                      'cloud admin', 'cloud solutions architect', 'cloud specialist', 'cloud devops'],
    
    'Kỹ sư phần mềm': ['software engineer', 'kỹ sư phần mềm', 'kỹ thuật phần mềm', 'swe', 'application engineer', 
                       'software dev', 'backend engineer', 'frontend engineer', 'software solutions engineer'],
    
    'Kỹ sư blockchain': ['blockchain engineer', 'kỹ sư blockchain', 'blockchain developer', 'smart contract developer', 
                         'crypto engineer', 'dapp developer', 'web3 developer', 'blockchain architect'],
    
    'Kỹ sư cơ sở dữ liệu': ['database engineer', 'cơ sở dữ liệu', 'database admin', 'dba', 'sql engineer', 'mysql admin', 
                            'oracle dba', 'postgresql engineer', 'database architect', 'data warehouse specialist'],
    
    'Kỹ sư trí tuệ nhân tạo': ['ai engineer', 'kỹ sư trí tuệ nhân tạo', 'artificial intelligence', 'ai architect', 
                               'ai developer', 'machine intelligence engineer', 'artificial intelligence scientist'],
    
    'Kỹ sư kiểm thử phần mềm': ['software tester', 'kiểm thử', 'test engineer', 'qa tester', 'quality assurance', 
                                'qa engineer', 'automation tester', 'manual tester', 'test analyst', 'sdet', 'software qa'],

    'Công nghệ thông tin (tổng quát)': ['it', 'công nghệ thông tin', 'cntt', 'information technology', 'it specialist', 
                                       'it technician', 'it officer', 'it expert', 'it solutions', 'it professional'],
}


# Hàm phân loại
def analyze_job_name(job: str):
    job = job.lower().strip()  # Chuyển về chữ thường và loại bỏ khoảng trắng
    for category, keywords in job_keywords.items():
        match = process.extractOne(job, keywords, scorer=fuzz.partial_ratio)
        if match and match[1] > 80:  # Ngưỡng độ tương đồng > 80%
            return category
    return job  # Trường hợp không khớp với bất kỳ công việc nào

## Ngày đăng tuyển

In [40]:
def analyze_posted_date(time: str):
    # Làm sạch dữ liệu đầu vào
    time = time.replace('một', '1')
    words = time.split()
    # Tìm vị trí của từ bắt đầu bằng số
    for i, word in enumerate(words):
        if word.isdigit():
            break
    if not words:
        return None  # Chuỗi không hợp lệ
    
    time_value = int(words[i])
    time_unit = words[i+1]

    # Lấy thời điểm hiện tại
    now = datetime.datetime.now()

    # Tính toán thời điểm tương ứng
    if time_unit == 'giây':
        result = now - datetime.timedelta(seconds=time_value)
    elif time_unit == 'phút':
        result = now - datetime.timedelta(minutes=time_value)
    elif time_unit == 'giờ':
        result = now - datetime.timedelta(hours=time_value)
    elif time_unit == 'ngày':
        result = now - datetime.timedelta(days=time_value)
    elif time_unit == 'tháng':
        result = now.replace(month=now.month - time_value)
    elif time_unit == 'năm':
        result = now.replace(year=now.year - time_value)
    else:
        return None

    return result

## Lương

In [41]:
def analyze_salary(salary: str):
    salary = salary.lower().strip()
    
    if salary == 'thương lượng':
        return None, None, None  # Thương lượng: cả 3 cột để trống
    
    elif salary == 'cạnh tranh':
        return 0, 0, 0  # Cạnh tranh: cả 3 cột bằng 0
    
    elif 'triệu' in salary:
        if 'trên' in salary:
            # Định dạng 'Trên X triệu'
            min_salary = int(float(salary.split('trên')[1].strip().replace('triệu', '').strip()) * 1_000_000)
            return min_salary, None, None
        elif '-' in salary:
            # Định dạng 'X triệu - Y triệu'
            min_salary, max_salary = map(
                lambda x: int(float(x.strip().replace('triệu', '').replace(',', '.')) * 1_000_000),
                salary.split('-')
            )
            avg_salary = (min_salary + max_salary) // 2
            return min_salary, max_salary, avg_salary
        else:
            # Định dạng 'X triệu'
            avg_salary = int(float(salary.replace('triệu', '').replace(',', '.').strip()) * 1_000_000)
            return None, None, avg_salary
        
    elif 'usd' in salary:
        if '-' in salary:
            # Định dạng 'X USD - Y USD', chuyển sang VNĐ
            min_salary, max_salary = map(
                lambda x: int(math.floor(float(x.strip().replace('usd', '').replace(',', '').strip()) * 25_000 / 1_000_000) * 1_000_000),
                salary.split('-')
            )
            avg_salary = (min_salary + max_salary) // 2
            return min_salary, max_salary, avg_salary
        else:
            # Định dạng 'X USD'
            avg_salary = int(math.floor(float(salary.replace('usd', '').replace(',', '').strip()) * 25_000 / 1_000_000) * 1_000_000)
            return None, None, avg_salary
    
    else:
        return None, None, None  # Nếu không khớp định dạng, để trống

## Yêu cầu bằng cấp

In [42]:
def degree_assessment(degree: str):
    degree = degree.lower().strip()  # Chuyển về chữ thường và loại bỏ khoảng trắng đầu/cuối
    if 'đại học' in degree or 'cao đẳng' in degree:
        return 'Cử nhân'
    elif 'trung học' in degree:
        return 'Trung học'
    elif 'trung cấp' in degree:
        return 'Trung cấp'
    else:
        return degree

## Yêu cầu kinh nghiệm

In [43]:
def level_assessment(level: str):
    # Kiểm tra 'Chưa có kinh nghiệm'
    if 'Chưa có kinh nghiệm' in level:
        return 'Fresher'

    # Kiểm tra 'Dưới X năm'
    elif 'Dưới' in level:
        return 'Fresher'

    # Kiểm tra 'Trên X năm'
    elif 'Trên' in level or 'Hơn' in level:
        return 'Senior'

    elif '-' in level:
        # Định dạng 'X - Y năm kinh nghiệm'
        match = re.match(r'(\d+)\s*-\s*(\d+)\s*năm\s*', level)
        x = int(match.group(1))
        y = int(match.group(2))
        avg = (x + y) / 2
        
    else:
        # Định dạng 'X năm'
        match = re.match(r'(\d+)\s*năm\s*', level)
        if match:
            avg = int(match.group(1))

    # Đánh giá cấp độ
    if avg <= 1:
        return 'Fresher'
    elif 1 < avg < 3:
        return 'Junior'
    elif 3 <= avg < 5:
        return 'Middle'
    else:
        return 'Senior'

# Truy cập và trích xuất dữ liệu từ trang web

## CareerLink

In [44]:
try:
    # Mở trang web CareerLink
    driver.get('https://www.careerlink.vn/viec-lam/cntt-phan-mem/19')

    # Trích xuất các công việc
    wait = WebDriverWait(driver, 10)
    jobs = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'list-group-item')))

    for job in jobs:
        try:
            job_text = job.text.split('\n')  # Tách thông tin thành từng dòng
            if len(job_text) < 5:  # Bỏ qua các mục không hợp lệ
                continue
            index_save = next((i for i, line in enumerate(job_text) if 'Lưu' in line), None)  # Bỏ các dòng 'Lưu' và trên nó
            if index_save is not None:
                job_text = job_text[index_save + 1:]

            # Click vào công việc để chuyển đến trang thông tin chi tiết của công việc đó
            job_detail_link = job.find_element(By.CSS_SELECTOR, 'a.job-link.clickable-outside')
            job_detail_link.click()

           # Chuyển sang tab mới
            window_handles = driver.window_handles
            driver.switch_to.window(window_handles[-1])

            # Thu thập yêu cầu kinh nghiệm
            experience_required_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.job-overview.mt-2'))
            )
            experience_required_text = experience_required_element.find_elements(By.XPATH, './*')[-2].find_element(By.CSS_SELECTOR, 'span').text

            # Trở lại trang chủ
            if len(window_handles) > 1:  # Đảm bảo có ít nhất 2 window (window chính + 1 window mới)
                driver.close()
                driver.switch_to.window(window_handles[0])

            # Tổng hợp các thông tin thu thập được
            website_name = 'CareerLink'
            job_name = analyze_job_name(job_text[0])
            company_name = job_text[1]
            place = job_text[2]
            posted_date = analyze_posted_date(job_text[3])
            salary = analyze_salary(job_text[4])
            position = job_text[6]
            experience_required = level_assessment(experience_required_text)

            # Thêm dữ liệu vào DataFrame
            new_row = {
                'Trang thu thập': 'CareerLink',
                'Tên công ty': company_name,
                'Tên công việc': job_name,
                'Vị trí ứng tuyển': position,
                'Yêu cầu bằng cấp': None,
                'Yêu cầu kinh nghiệm': experience_required,
                'Địa điểm': place,
                'Ngày đăng tuyển': posted_date,
                'Lương tối thiểu': salary[0],
                'Lương tối đa': salary[1],
                'Lương TB': salary[2]
            }
            df.loc[len(df)] = new_row
        
        except Exception as e:
            print(f'Lỗi khi xử lý công việc: {e}')
            driver.close()
            driver.switch_to.window(driver.window_handles[0])  # Đảm bảo quay lại trang chính nếu có lỗi

except Exception as e:
    print(f'Có lỗi xảy ra trong quá trình thu thập dữ liệu: {e}')

## việclàm24h

In [ ]:
try:
    # Mở trang web việclàm24h
    driver.get('https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?occupation_ids[]=8&occupation_ids[]=7&sort_q=priority_max%2Cdesc')

    # Trích xuất các công việc
    wait = WebDriverWait(driver, 10)
    jobs = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'flex.flex-col.gap-3')))[0].find_elements(By.XPATH, './*')
    c = 0
    
    for job in jobs:
        try:
            job_text = job.text.split('\n')  # Tách thông tin thành từng dòng

            # Click vào công việc để chuyển đến trang thông tin chi tiết của công việc đó
            job.click()
            
            # Chuyển sang tab mới
            window_handles = driver.window_handles
            driver.switch_to.window(window_handles[-1])

            # Tìm các thông tin chi tiết
            job_details = WebDriverWait(driver, 10).until(
                EC.visibility_of_all_elements_located((By.XPATH, "//div[@id='__next']/div/main/div/div[3]/div/div[2]/div/div/div/h3"))
            )
            job_detail_text = [item for job_detail in job_details for item in job_detail.text.split('\n')]
            
            # Tổng hợp các thông tin thu thập được
            website_name = 'vieclam24h'
            job_name = analyze_job_name(job_text[0])
            company_name = job_text[1]
            salary = analyze_salary(job_text[2])
            place = job_text[3]
            posted_date = datetime.datetime.strptime(job_detail_text[job_detail_text.index('Ngày đăng') + 1], '%d/%m/%Y')
            position = job_detail_text[job_detail_text.index('Cấp bậc') + 1]
            if 'Yêu cầu bằng cấp' in job_detail_text:
                degree_required = degree_assessment(job_detail_text[job_detail_text.index('Yêu cầu bằng cấp') + 1])
            else:
                degree_required = None
            experience_required = level_assessment(job_detail_text[job_detail_text.index('Yêu cầu kinh nghiệm') + 1])
            
            # Trở lại trang chủ
            if len(window_handles) > 1:  # Đảm bảo có ít nhất 2 window (window chính + 1 window mới)
                driver.close()
                driver.switch_to.window(window_handles[0])
            
            # Thêm dữ liệu vào DataFrame
            new_row = {
                'Trang thu thập': 'CareerLink',
                'Tên công ty': company_name,
                'Tên công việc': job_name,
                'Vị trí ứng tuyển': position,
                'Yêu cầu bằng cấp': degree_required,
                'Yêu cầu kinh nghiệm': experience_required,
                'Địa điểm': place,
                'Ngày đăng tuyển': posted_date,
                'Lương tối thiểu': salary[0],
                'Lương tối đa': salary[1],
                'Lương TB': salary[2]
            }
            df.loc[len(df)] = new_row

        except Exception as e:
            print(f'Lỗi khi xử lý công việc: {e}')
            driver.close()
            driver.switch_to.window(driver.window_handles[0])  # Đảm bảo quay lại trang chính nếu có lỗi
        
except Exception as e:
    print(f'Có lỗi xảy ra trong quá trình thu thập dữ liệu: {e}')

# Lưu DataFrame vào file csv

In [ ]:
# In da DataFrame
df.to_csv('job_data.csv', index=True, encoding='utf-8')
df

Đóng trình duyệt:

In [47]:
driver.quit()